In [1]:
import pandas as pd 
import numpy as np
import xgboost as xgb 
from prediction_utils import knapsack_fpl_xi

/Users/terryzhou/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Update Raw Player Data!!!

Update `data/2023-24/cleaned_players.csv` by running the following in terminal: 
1. `git fetch upstream master`
2. `git checkout upstream/master -- data/2023-24/cleaned_players.csv`

In [2]:
# The game week making prediction for 
current_gw = 5

# Import models and data

In [3]:
df = pd.read_csv("../../data/cleaned_data/2024_cleaned_prediction_data.csv")
df = df[df["minutes"] >= 60.0]
df_current = df[df["GW"]==current_gw-1]
df_current.head()

,name,position,team,GW,minutes,goals_scored,assists,clean_sheets,saves,penalties_saved,...,goals_conceded,yellow_cards,red_cards,own_goals,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,team_standing,sched_strength
2,Aaron Hickey,DEF,Brentford,4,79.75,0.00,0.00,0.25,0.00,0.0,...,1.25,0.50,0.0,0.0,0.0175,0.0300,0.0475,1.0500,7.25,14.4
3,Aaron Ramsdale,GK,Arsenal,4,90.00,0.00,0.00,0.25,1.25,0.0,...,1.00,0.00,0.0,0.0,0.0000,0.0025,0.0025,0.9325,4.25,9.8
5,Aaron Wan-Bissaka,DEF,Man Utd,4,83.75,0.00,0.25,0.25,0.00,0.0,...,1.50,0.50,0.0,0.0,0.0000,0.2325,0.2325,1.7575,9.00,11.6
6,Abdoulaye Doucouré,MID,Everton,4,90.00,0.25,0.00,0.00,0.00,0.0,...,2.00,0.25,0.0,0.0,0.4825,0.0650,0.5475,1.7525,18.25,10.2
25,Alexander Isak,FWD,Newcastle,4,69.00,0.50,0.00,0.25,0.00,0.0,...,1.25,0.00,0.0,0.0,0.3500,0.0225,0.3725,1.0075,9.00,11.2


In [4]:
fwd_model = xgb.XGBRegressor()
fwd_model.load_model("../../data/models/fwd_model.json")
mid_model = xgb.XGBRegressor()
mid_model.load_model("../../data/models/mid_model.json")
def_model = xgb.XGBRegressor()
def_model.load_model("../../data/models/def_model.json")
gk_model = xgb.XGBRegressor()
gk_model.load_model("../../data/models/gk_model.json")

# Attach cost to players 

In [5]:
df_players = pd.read_csv("../../data/2023-24/cleaned_players.csv")
df_players["name"] = df_players.apply(lambda row: row["first_name"] + " " + row["second_name"], axis=1)
df_players = df_players[["name", "now_cost"]]
df_players.head()

,name,now_cost
0,Folarin Balogun,44
1,Cédric Alves Soares,40
2,Mohamed Elneny,44
3,Fábio Ferreira Vieira,54
4,Gabriel dos Santos Magalhães,48


In [6]:
df_current = pd.merge(df_current, df_players, how="left", on="name")
df_current = df_current[df_current["now_cost"].notna()]
df_current.head()

,name,position,team,GW,minutes,goals_scored,assists,clean_sheets,saves,penalties_saved,...,yellow_cards,red_cards,own_goals,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,team_standing,sched_strength,now_cost
0,Aaron Hickey,DEF,Brentford,4,79.75,0.00,0.00,0.25,0.00,0.0,...,0.50,0.0,0.0,0.0175,0.0300,0.0475,1.0500,7.25,14.4,45
1,Aaron Ramsdale,GK,Arsenal,4,90.00,0.00,0.00,0.25,1.25,0.0,...,0.00,0.0,0.0,0.0000,0.0025,0.0025,0.9325,4.25,9.8,50
2,Aaron Wan-Bissaka,DEF,Man Utd,4,83.75,0.00,0.25,0.25,0.00,0.0,...,0.50,0.0,0.0,0.0000,0.2325,0.2325,1.7575,9.00,11.6,46
3,Abdoulaye Doucouré,MID,Everton,4,90.00,0.25,0.00,0.00,0.00,0.0,...,0.25,0.0,0.0,0.4825,0.0650,0.5475,1.7525,18.25,10.2,55
4,Alexander Isak,FWD,Newcastle,4,69.00,0.50,0.00,0.25,0.00,0.0,...,0.00,0.0,0.0,0.3500,0.0225,0.3725,1.0075,9.00,11.2,77


# Predict FPL 5-game avg points using models

## Running Model

In [7]:
df_fwd = df_current[df_current["position"] == "FWD"]
df_mid = df_current[df_current["position"] == "MID"]
df_def = df_current[df_current["position"] == "DEF"] 
df_gk = df_current[df_current["position"] == "GK"] 

In [8]:
df_fwd_input = df_fwd.drop(["name", "position", "team", "now_cost"], axis=1) 
df_mid_input = df_mid.drop(["name", "position", "team", "now_cost"], axis=1) 
df_def_input = df_def.drop(["name", "position", "team", "now_cost"], axis=1) 
df_gk_input = df_gk.drop(["name", "position", "team", "now_cost"], axis=1) 

In [9]:
df_fwd["xP"] = fwd_model.predict(df_fwd_input)
df_mid["xP"] = mid_model.predict(df_mid_input)
df_def["xP"] = def_model.predict(df_def_input)
df_gk["xP"] = gk_model.predict(df_gk_input)

/var/folders/64/lts2my7j2q3g96y6cn7715xw0000gn/T/ipykernel_86657/678547785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fwd["xP"] = fwd_model.predict(df_fwd_input)
/var/folders/64/lts2my7j2q3g96y6cn7715xw0000gn/T/ipykernel_86657/678547785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mid["xP"] = mid_model.predict(df_mid_input)
/var/folders/64/lts2my7j2q3g96y6cn7715xw0000gn/T/ipykernel_86657/678547785.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [10]:
df_fwd_out = df_fwd[["name", "position", "team", "now_cost", "xP"]].sort_values("xP", ascending=False).reset_index(drop=True)
df_mid_out = df_mid[["name", "position", "team", "now_cost", "xP"]].sort_values("xP", ascending=False).reset_index(drop=True)
df_def_out = df_def[["name", "position", "team", "now_cost", "xP"]].sort_values("xP", ascending=False).reset_index(drop=True)
df_gk_out = df_gk[["name", "position", "team", "now_cost", "xP"]].sort_values("xP", ascending=False).reset_index(drop=True)

## Player Rankings 

In [11]:
df_fwd_out.head(10)

,name,position,team,now_cost,xP
0,Yoane Wissa,FWD,Brentford,62,4.545316
1,Michail Antonio,FWD,West Ham,61,4.258509
2,Erling Haaland,FWD,Man City,141,3.618991
3,Odsonne Edouard,FWD,Crystal Palace,55,3.359924
4,Julián Álvarez,FWD,Man City,67,3.318550
5,Taiwo Awoniyi,FWD,Nott'm Forest,66,3.132548
6,Eddie Nketiah,FWD,Arsenal,56,2.970721
7,Alexander Isak,FWD,Newcastle,77,2.880027
8,Raúl Jiménez,FWD,Fulham,54,2.765237
9,Nicolas Jackson,FWD,Chelsea,72,2.584388


In [12]:
df_mid_out.head(10)

,name,position,team,now_cost,xP
0,Son Heung-min,MID,Spurs,91,6.039016
1,Martin Ødegaard,MID,Arsenal,85,5.948623
2,Mohamed Salah,MID,Liverpool,125,5.541354
3,Bryan Mbeumo,MID,Brentford,68,5.438502
4,Dejan Kulusevski,MID,Spurs,70,5.171972
5,James Maddison,MID,Spurs,78,4.933184
6,Rodrigo Hernandez,MID,Man City,57,4.807833
7,Pascal Groß,MID,Brighton,64,4.318693
8,Solly March,MID,Brighton,66,4.147378
9,Kai Havertz,MID,Arsenal,73,4.088475


In [13]:
df_def_out.head(10)

,name,position,team,now_cost,xP
0,Tyrick Mitchell,DEF,Crystal Palace,45,4.070144
1,Joachim Andersen,DEF,Crystal Palace,46,3.895018
2,Marc Guéhi,DEF,Crystal Palace,45,3.555871
3,Micky van de Ven,DEF,Spurs,45,3.396158
4,Aaron Hickey,DEF,Brentford,45,3.393091
5,Destiny Udogie,DEF,Spurs,47,3.388441
6,Kurt Zouma,DEF,West Ham,45,3.222249
7,Ethan Pinnock,DEF,Brentford,45,3.148459
8,Andrew Robertson,DEF,Liverpool,65,3.017778
9,James Tarkowski,DEF,Everton,45,3.011088


In [14]:
df_gk_out.head()

,name,position,team,now_cost,xP
0,Jordan Pickford,GK,Everton,45,3.970370
1,James Trafford,GK,Burnley,45,3.628549
2,Alisson Ramses Becker,GK,Liverpool,55,3.558679
3,Aaron Ramsdale,GK,Arsenal,50,3.505435
4,Thomas Kaminski,GK,Luton,45,3.419714


# Generate recommended starting XI 

In [19]:
budget = 835
fwd_count = 2
mid_count = 5
def_count = 3
gk_count = 1

df_players = pd.concat([df_fwd_out, df_mid_out, df_def_out, df_gk_out], ignore_index=True) 
df_players["now_cost"] = df_players["now_cost"].astype("int")

In [20]:
df_selected = knapsack_fpl_xi(df_players, budget, fwd_count, mid_count, def_count, gk_count)
df_selected

,name,position,team,now_cost,xP
0,Yoane Wissa,FWD,Brentford,62,4.545316
1,Michail Antonio,FWD,West Ham,61,4.258509
19,Son Heung-min,MID,Spurs,91,6.039016
20,Martin Ødegaard,MID,Arsenal,85,5.948623
21,Mohamed Salah,MID,Liverpool,125,5.541354
22,Bryan Mbeumo,MID,Brentford,68,5.438502
23,Dejan Kulusevski,MID,Spurs,70,5.171972
94,Tyrick Mitchell,DEF,Crystal Palace,45,4.070144
95,Joachim Andersen,DEF,Crystal Palace,46,3.895018
96,Marc Guéhi,DEF,Crystal Palace,45,3.555871


In [21]:
print(f"total_xP_per_gw: {df_selected.xP.sum()}")
print(f"total_xP_over_5_gw: {df_selected.xP.sum() * 5}")
print(f"total_cost: {df_selected.now_cost.sum()}")

total_xP_per_gw: 52.434696197509766
total_xP_over_5_gw: 262.1734809875488
total_cost: 743
